In [2]:
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt
import bitstring
from qiskit_aer import AerSimulator

In [3]:
def rot_surf_code(n: int) -> QuantumCircuit:              #first ancilla = second last qubit = syndrome/parity measurement qubit, secondt ancilla = last qubit = magic state for T Gate
    qr = QuantumRegister(9*n+2, "q")
    cbit = ClassicalRegister(4,"c")
    qc = QuantumCircuit(qr,cbit)
    list = []
    for i in range(0, 9*n):
        list.append("{:c}".format(i))
    list = qr
    for i in range(n):
        qc.h(list[9*i+1])
        qc.h(list[9*i+3])
        qc.h(list[9*i+5])
        qc.h(list[9*i+7])

        qc.cx(list[9*i+1],list[9*i])
        qc.cx(list[9*i+5],list[9*i+4])
        qc.cx(list[9*i+7],list[9*i+8])

        qc.cx(list[9*i+5],list[9*i+2])

        qc.cx(list[9*i+3],list[9*i+4])
        qc.cx(list[9*i+2],list[9*i+1])

        qc.cx(list[9*i+3],list[9*i+6])

        qc.cx(list[9*i+6],list[9*i+7])
    return qc

In [47]:
def X_L(circuit: QuantumCircuit, hadamard = False, n=0):            #n muss bei 0 anfangen, also z.B. man hat 3 Circuits und will X_L auf den ersten anwenden ---> n = 0 !!!
    if hadamard:
        circuit.x(9*n+0)
        circuit.x(9*n+1)
        circuit.x(9*n+2)
    else:
        circuit.x(9*n+0)
        circuit.x(9*n+3)
        circuit.x(9*n+6)
    return circuit

def Z_L(circuit: QuantumCircuit, hadamard = False, n=0):            #n muss bei 0 anfangen, also z.B. man hat 3 Circuits und will Z_L auf den ersten anwenden ---> n = 0 !!!
    if hadamard:
        circuit.z(9*n+2)
        circuit.z(9*n+5)
        circuit.z(9*n+8)
    else:
        circuit.z(9*n+6)
        circuit.z(9*n+7)
        circuit.z(9*n+8)
    return circuit

def Hadamard(circuit: QuantumCircuit, pos=0):              #H-Gates auf EINEN Circuit, wieder erster Circuit --> n = 0, zweiter Circuit --> n = 1 , usw.
    for i in range(9):
        circuit.h(9*pos+i)
    return circuit

def S_L(circuit: QuantumCircuit, hadamard = False, pos=0):
    for i in range(9):
        circuit.s(9*pos+i)
    qc = Z_L(circuit, hadamard=hadamard, n = pos)
    return qc

def adjoint_S_L(circuit: QuantumCircuit, hadamard = False, pos=0):
    for i in range(9):
        circuit.sdg(9*pos+i)
    qc = Z_L(circuit, hadamard=hadamard, n = pos)
    return qc

def Z_parity(circuit: QuantumCircuit, hadamard = False, pos = 0):
    qc = circuit
    ancilla = qc.num_qubits - 2
    if hadamard:
        qc.cx(1+9*pos,ancilla)
        qc.cx(4+9*pos,ancilla)
        qc.cx(7+9*pos,ancilla)
    else:
        qc.cx(3+9*pos,ancilla)
        qc.cx(4+9*pos,ancilla)
        qc.cx(5+9*pos,ancilla)
    return qc

def X_parity(circuit: QuantumCircuit, hadamard = False, pos = 0):
    qc = circuit
    ancilla = qc.num_qubits - 2
    #qc.h(ancilla)
    if hadamard:
        qc.cx(ancilla,3+9*pos)
        qc.cx(ancilla,4+9*pos)
        qc.cx(ancilla,5+9*pos)
    else:
        qc.cx(ancilla,1+9*pos)
        qc.cx(ancilla,4+9*pos)
        qc.cx(ancilla,7+9*pos)
    #qc.h(ancilla)
    return qc

def reset_logical_qubit(qc: QuantumCircuit, pos = 0):
    for i in range(9):
        qc.reset(9*pos+i)
    i = pos
    qc.h(9*i+1)
    qc.h(9*i+3)
    qc.h(9*i+5)
    qc.h(9*i+7)

    qc.cx(9*i+1, 9*i)
    qc.cx(9*i+5, 9*i+4)
    qc.cx(9*i+7, 9*i+8)

    qc.cx(9*i+5, 9*i+2)

    qc.cx(9*i+3, 9*i+4)
    qc.cx(9*i+2, 9*i+1)

    qc.cx(9*i+3,9*i+6)

    qc.cx(9*i+6,9*i+7)
    return qc
    
def T_L(circuit: QuantumCircuit, had = False, pos = 0):
    qc = circuit
    magic = qc.num_qubits - 1
    ancilla = qc.num_qubits - 2
    qc.reset(magic)
    qc.reset(ancilla)
    
    qc.h(magic)
    qc.t(magic)                 #Initliaisierung vom Magic State

    qc = Z_parity(qc, hadamard=False, pos = pos)

    qc.cx(magic, ancilla)

    qc.measure(ancilla,0)
    
    if had == False:
        with qc.if_test((0,1)):
            qc.s(0+9*pos)
            qc.s(1+9*pos)
            qc.s(2+9*pos)
            qc.s(3+9*pos)
            qc.s(4+9*pos)
            qc.s(5+9*pos)
            qc.s(6+9*pos)
            qc.s(7+9*pos)
            qc.s(8+9*pos)

            qc.z(3+9*pos)
            qc.z(4+9*pos)
            qc.z(5+9*pos)
    else:
        with qc.if_test((0,1)):
            qc.s(0+9*pos)
            qc.s(1+9*pos)
            qc.s(2+9*pos)
            qc.s(3+9*pos)
            qc.s(4+9*pos)
            qc.s(5+9*pos)
            qc.s(6+9*pos)
            qc.s(7+9*pos)
            qc.s(8+9*pos)
            
            qc.z(1+9*pos)
            qc.z(4+9*pos)
            qc.z(7+9*pos)
    
    qc.h(magic)
    qc.measure(magic,0)
    
    if had == False:
        with qc.if_test((0,1)):
            qc.z(3+9*pos)
            qc.z(4+9*pos)
            qc.z(5+9*pos)
    else:
        with qc.if_test((0,1)):
            qc.z(1+9*pos)
            qc.z(4+9*pos)
            qc.z(7+9*pos)
    return qc

def adjoint_T_L(circuit: QuantumCircuit, had = False, pos = 0):
    qc = circuit
    magic = qc.num_qubits - 1
    ancilla = qc.num_qubits - 2
    qc.reset(magic)
    qc.reset(ancilla)
    
    qc.h(magic)
    qc.tdg(magic)                 #Initliaisierung vom Magic State

    qc = Z_parity(qc, hadamard=False, pos = pos)
    qc.cx(magic, ancilla)

    qc.measure(ancilla,0)
    
    if had == False:
        with qc.if_test((0,1)):
            qc.sdg(0+9*pos)
            qc.sdg(1+9*pos)
            qc.sdg(2+9*pos)
            qc.sdg(3+9*pos)
            qc.sdg(4+9*pos)
            qc.sdg(5+9*pos)
            qc.sdg(6+9*pos)
            qc.sdg(7+9*pos)
            qc.sdg(8+9*pos)

            qc.z(3+9*pos)
            qc.z(4+9*pos)
            qc.z(5+9*pos)
    else:
        with qc.if_test((0,1)):
            qc.sdg(0+9*pos)
            qc.sdg(1+9*pos)
            qc.sdg(2+9*pos)
            qc.sdg(3+9*pos)
            qc.sdg(4+9*pos)
            qc.sdg(5+9*pos)
            qc.sdg(6+9*pos)
            qc.sdg(7+9*pos)
            qc.sdg(8+9*pos)
            
            qc.z(1+9*pos)
            qc.z(4+9*pos)
            qc.z(7+9*pos)
    
    qc.h(magic)
    qc.measure(magic,0)
    
    if had == False:
        with qc.if_test((0,1)):
            qc.z(3+9*pos)
            qc.z(4+9*pos)
            qc.z(5+9*pos)
    else:
        with qc.if_test((0,1)):
            qc.z(1+9*pos)
            qc.z(4+9*pos)
            qc.z(7+9*pos)
    return qc

def CNOT(circuit:QuantumCircuit, hadamard = "00", control = 0):               #CNOT mit berücksichtigung der Rotation durch H-Gate
    if control == 0:
        if hadamard == "10":
            circuit.cx(control_qubit=0,target_qubit=9+6)
            circuit.cx(control_qubit=1,target_qubit=9+3)
            circuit.cx(control_qubit=2,target_qubit=9+0)
            circuit.cx(control_qubit=3,target_qubit=9+7)
            circuit.cx(control_qubit=4,target_qubit=9+4)
            circuit.cx(control_qubit=5,target_qubit=9+1)
            circuit.cx(control_qubit=6,target_qubit=9+8)
            circuit.cx(control_qubit=7,target_qubit=9+5)
            circuit.cx(control_qubit=8,target_qubit=9+2)
        elif hadamard == "01":
            circuit.cx(control_qubit=0,target_qubit=9+2)
            circuit.cx(control_qubit=1,target_qubit=9+5)
            circuit.cx(control_qubit=2,target_qubit=9+8)
            circuit.cx(control_qubit=3,target_qubit=9+1)
            circuit.cx(control_qubit=4,target_qubit=9+4)
            circuit.cx(control_qubit=5,target_qubit=9+7)
            circuit.cx(control_qubit=6,target_qubit=9+0)
            circuit.cx(control_qubit=7,target_qubit=9+3)
            circuit.cx(control_qubit=8,target_qubit=9+6)
        else:
            for i in range(9):
                circuit.cx(control_qubit=i,target_qubit=9+i)
    elif control == 1:
        if hadamard == "01":
            circuit.cx(control_qubit=9+0,target_qubit=6)
            circuit.cx(control_qubit=9+1,target_qubit=3)
            circuit.cx(control_qubit=9+2,target_qubit=0)
            circuit.cx(control_qubit=9+3,target_qubit=7)
            circuit.cx(control_qubit=9+4,target_qubit=4)
            circuit.cx(control_qubit=9+5,target_qubit=1)
            circuit.cx(control_qubit=9+6,target_qubit=8)
            circuit.cx(control_qubit=9+7,target_qubit=5)
            circuit.cx(control_qubit=9+8,target_qubit=2)
        elif hadamard == "10":
            circuit.cx(control_qubit=9+0,target_qubit=2)
            circuit.cx(control_qubit=9+1,target_qubit=5)
            circuit.cx(control_qubit=9+2,target_qubit=8)
            circuit.cx(control_qubit=9+3,target_qubit=1)
            circuit.cx(control_qubit=9+4,target_qubit=4)
            circuit.cx(control_qubit=9+5,target_qubit=7)
            circuit.cx(control_qubit=9+6,target_qubit=0)
            circuit.cx(control_qubit=9+7,target_qubit=3)
            circuit.cx(control_qubit=9+8,target_qubit=6)
        else: 
            for i in range(9):
                circuit.cx(control_qubit=9+i,target_qubit=i)
    return circuit

def control_S_L(circuit: QuantumCircuit, hadamard = "00", control = 0, target = 1):
    qc = circuit
    qc = T_L(qc, had=False, pos=control)
    qc = T_L(qc, had=True, pos=target)
    qc = CNOT(qc, hadamard=hadamard,control=control)
    qc = adjoint_T_L(qc,had=True,pos=target)
    qc = CNOT(qc, hadamard=hadamard,control=control)
    return qc

def control_Z_L(circuit: QuantumCircuit, control = 0, target = 1):
    qc = circuit
    qc = Hadamard(qc, pos = target)
    qc = CNOT(qc, control = control)            #aufgrund des H eine Zeile drüber, geht das normale CNOT
    qc = Hadamard(qc, pos = target)
    return qc

def control_Z_L_test(circuit: QuantumCircuit, hadamard = "00", control = 0, target = 1):
    qc = circuit
    qc = S_L(qc, hadamard=False, pos=control)
    qc = S_L(qc, hadamard=True, pos=target)
    qc = CNOT(qc, hadamard=hadamard,control=control)
    qc = adjoint_S_L(qc,hadamard=True,pos=target)
    qc = CNOT(qc, hadamard=hadamard,control=control)
    return qc

def stabilize(circuit: QuantumCircuit, hadamard = False, pos = 0):                     #misst alle stabilizer, speichert den Fehler und korrigiert dementsprechend
    qc = circuit
    ancilla = qc.num_qubits - 2
    if hadamard:

        #X3 X6 Stabilizer:
        qc.reset(ancilla)
        qc.h(ancilla)
        qc.cx(ancilla,3+9*pos)
        qc.cx(ancilla,6+9*pos)
        qc.h(ancilla)
        qc.measure(ancilla,0)

        #X0 X1 X3 X4 Stabilizer:
        qc.reset(ancilla)
        qc.h(ancilla)
        qc.cx(ancilla,0+9*pos)
        qc.cx(ancilla,1+9*pos)
        qc.cx(ancilla,3+9*pos)
        qc.cx(ancilla,4+9*pos)
        qc.h(ancilla)
        qc.measure(ancilla,1)

        #X4 X5 X7 X8 Stabilizer:
        qc.reset(ancilla)
        qc.h(ancilla)
        qc.cx(ancilla,4+9*pos)
        qc.cx(ancilla,5+9*pos)
        qc.cx(ancilla,7+9*pos)
        qc.cx(ancilla,8+9*pos)
        qc.h(ancilla)
        qc.measure(ancilla,2)

        #X2 X5 Stabilizer:
        qc.reset(ancilla)
        qc.h(ancilla)
        qc.cx(ancilla,2+9*pos)
        qc.cx(ancilla,5+9*pos)
        qc.h(ancilla)
        qc.measure(ancilla,3)

        with qc.if_test((0,1)):             #6
            with qc.if_test((1,0)):
                qc.z(6+9*pos)

        with qc.if_test((0,1)):             #3
            with qc.if_test((1,1)):
                qc.z(3+9*pos)

        with qc.if_test((3,1)):             #2
            with qc.if_test((2,0)):
                qc.z(2+9*pos)
        
        with qc.if_test((3,1)):             #5
            with qc.if_test((2,1)):
                qc.z(5+9*pos)
        
        with qc.if_test((1,1)):             #4
            with qc.if_test((2,1)):
                qc.z(4+9*pos)

        with qc.if_test((0,0)):             #0 und 1
            with qc.if_test((1,1)):
                with qc.if_test((2,0)):
                    qc.z(0+9*pos)
        
        with qc.if_test((1,0)):             #7 und 8
            with qc.if_test((2,1)):
                with qc.if_test((3,0)):
                    qc.z(7+9*pos)

    ###########################################################################################################

        #Z0 Z1 Stabilizer:
        qc.reset(ancilla)
        qc.cx(0+9*pos,ancilla)
        qc.cx(1+9*pos,ancilla)
        qc.measure(ancilla,0)

        #Z1 Z2 Z4 Z5 Stabilizer:
        qc.reset(ancilla)
        qc.cx(1+9*pos,ancilla)
        qc.cx(2+9*pos,ancilla)
        qc.cx(4+9*pos,ancilla)
        qc.cx(5+9*pos,ancilla)
        qc.measure(ancilla,1)
    
        #Z3 Z4 Z6 Z7 Stabilizer:
        qc.reset(ancilla)
        qc.cx(3+9*pos,ancilla)
        qc.cx(4+9*pos,ancilla)
        qc.cx(6+9*pos,ancilla)
        qc.cx(7+9*pos,ancilla)
        qc.measure(ancilla,2)

        #Z7 Z8 Stabilizer:
        qc.reset(ancilla)
        qc.cx(7+9*pos,ancilla)
        qc.cx(8+9*pos,ancilla)
        qc.measure(ancilla,3)
        
        with qc.if_test((0,1)):             #0
            with qc.if_test((1,0)):
                qc.x(0+9*pos)

        with qc.if_test((0,1)):             #1
            with qc.if_test((1,1)):
                qc.x(1+9*pos)
        
        with qc.if_test((3,1)):             #8
            with qc.if_test((2,0)):
                qc.x(8+9*pos)
        
        with qc.if_test((3,1)):             #7
            with qc.if_test((2,1)):
                qc.x(7+9*pos)
        
        with qc.if_test((1,1)):             #4
            with qc.if_test((2,1)):
                qc.x(4+9*pos)

        with qc.if_test((0,0)):             #2 und 5
            with qc.if_test((1,1)):
                with qc.if_test((2,0)):
                    qc.x(2+9*pos)
        
        with qc.if_test((1,0)):             #3 und 6
            with qc.if_test((2,1)):
                with qc.if_test((3,0)):
                    qc.x(3+9*pos)

    else:
        #X0 X1 Stabilizer:
        qc.reset(ancilla)
        qc.h(ancilla)
        qc.cx(ancilla,0+9*pos)
        qc.cx(ancilla,1+9*pos)
        qc.h(ancilla)
        qc.measure(ancilla,0)
        
        #X1 X2 X4 X5 Stabilizer:
        qc.reset(ancilla)
        qc.h(ancilla)
        qc.cx(ancilla,1+9*pos)
        qc.cx(ancilla,2+9*pos)
        qc.cx(ancilla,4+9*pos)
        qc.cx(ancilla,5+9*pos)
        qc.h(ancilla)
        qc.measure(ancilla,1)

        #X3 X4 X6 X7 Stabilizer:
        qc.reset(ancilla)
        qc.h(ancilla)
        qc.cx(ancilla,3+9*pos)
        qc.cx(ancilla,4+9*pos)
        qc.cx(ancilla,6+9*pos)
        qc.cx(ancilla,7+9*pos)
        qc.h(ancilla)
        qc.measure(ancilla,2)

        #X7 X8 Stabilizer:
        qc.reset(ancilla)
        qc.h(ancilla)
        qc.cx(ancilla,7+9*pos)
        qc.cx(ancilla,8+9*pos)
        qc.h(ancilla)
        qc.measure(ancilla,3)

        with qc.if_test((0,1)):             #0
            with qc.if_test((1,0)):    
                qc.z(0+9*pos)
        
        with qc.if_test((0,1)):             #1
            with qc.if_test((1,1)):
                qc.z(1+9*pos)

        with qc.if_test((1,1)):             #4
            with qc.if_test((2,1)):
                qc.z(4+9*pos)

        with qc.if_test((2,1)):             #7
            with qc.if_test((3,1)):
                qc.z(7+9*pos)

        with qc.if_test((2,0)):             #8
            with qc.if_test((3,1)):
                qc.z(8+9*pos)

        with qc.if_test((0,0)):             #2 und 5
            with qc.if_test((1,1)):        
                with qc.if_test((2,0)):    
                    qc.z(2+9*pos)

        with qc.if_test((1,0)):             #3 und 6
            with qc.if_test((2,1)):
                with qc.if_test((3,0)):
                    qc.z(3+9*pos)

    ###########################################################################################################

        #Z3 Z6 Stabilizer:
        qc.reset(ancilla)
        qc.cx(3+9*pos,ancilla)
        qc.cx(6+9*pos,ancilla)
        qc.measure(ancilla,0)

        #Z0 Z1 Z3 Z4 Stabilizer:
        qc.reset(ancilla)
        qc.cx(0+9*pos,ancilla)
        qc.cx(1+9*pos,ancilla)
        qc.cx(3+9*pos,ancilla)
        qc.cx(4+9*pos,ancilla)
        qc.measure(ancilla,1)
    
        #Z4 Z5 Z7 Z8 Stabilizer:
        qc.reset(ancilla)
        qc.cx(4+9*pos,ancilla)
        qc.cx(5+9*pos,ancilla)
        qc.cx(7+9*pos,ancilla)
        qc.cx(8+9*pos,ancilla)
        qc.measure(ancilla,2)

        #Z2 Z5 Stabilizer:
        qc.reset(ancilla)
        qc.cx(2+9*pos,ancilla)
        qc.cx(5+9*pos,ancilla)
        qc.measure(ancilla,3)
        
        with qc.if_test((0,1)):             #6
            with qc.if_test((1,0)):
                qc.x(6+9*pos)

        with qc.if_test((0,1)):             #3
            with qc.if_test((1,1)):
                qc.x(3+9*pos)

        with qc.if_test((1,1)):             #4
            with qc.if_test((2,1)):
                qc.x(4+9*pos)

        with qc.if_test((2,0)):             #2
            with qc.if_test((3,1)):
                qc.x(2+9*pos)

        with qc.if_test((2,1)):             #5
            with qc.if_test((3,1)):
                qc.x(5+9*pos)
        
        with qc.if_test((0,0)):             #0 und 1
            with qc.if_test((1,1)):
                with qc.if_test((2,0)):
                    qc.x(0+9*pos)
        
        with qc.if_test((1,0)):             #7 und 8
            with qc.if_test((2,1)):
                with qc.if_test((3,0)):
                    qc.x(7+9*pos)
    return qc

def QEC(circuit: QuantumCircuit, hadamard = False, pos = 0, iterations = 1):
    for i in range(iterations):
        circuit = stabilize(circuit=circuit, hadamard=hadamard, pos=pos)
    return  circuit

def gates(circuit:QuantumCircuit):
    qc = circuit
    hmm = dict(qc.count_ops())
    return print("Amount of gates in this circuit: ", sum(hmm.values()))


In [55]:
def T_L_test(circuit: QuantumCircuit, had = False, pos = 0):
    qc = circuit
    magic = qc.num_qubits - 1
    ancilla = qc.num_qubits - 2
    qc.reset(magic)
    qc.reset(ancilla)
    
    qc.h(magic)
    qc.t(magic)                 #Initliaisierung vom Magic State

    qc = Z_parity(qc, hadamard=False, pos = pos)
    # if had:
    #     qc.cx(1+9*pos,ancilla)
    #     qc.cx(4+9*pos,ancilla)
    #     qc.cx(7+9*pos,ancilla)
    # else:
    #     qc.cx(3+9*pos,ancilla)
    #     qc.cx(4+9*pos,ancilla)
    #     qc.cx(5+9*pos,ancilla)

    qc.cx(magic, ancilla)

    qc.measure(ancilla,0)
    
    if had == False:
        with qc.if_test((0,1)):
            qc.s(0+9*pos)
            qc.s(1+9*pos)
            qc.s(2+9*pos)
            qc.s(3+9*pos)
            qc.s(4+9*pos)
            qc.s(5+9*pos)
            qc.s(6+9*pos)
            qc.s(7+9*pos)
            qc.s(8+9*pos)

            qc.z(6+9*pos)
            qc.z(7+9*pos)
            qc.z(8+9*pos)
    else:
        with qc.if_test((0,1)):
            qc.s(0+9*pos)
            qc.s(1+9*pos)
            qc.s(2+9*pos)
            qc.s(3+9*pos)
            qc.s(4+9*pos)
            qc.s(5+9*pos)
            qc.s(6+9*pos)
            qc.s(7+9*pos)
            qc.s(8+9*pos)
            
            qc.z(2+9*pos)
            qc.z(5+9*pos)
            qc.z(8+9*pos)
    
    qc.h(magic)
    qc.measure(magic,0)
    
    if had == False:
        with qc.if_test((0,1)):
            qc.z(6+9*pos)
            qc.z(7+9*pos)
            qc.z(8+9*pos)
    else:
        with qc.if_test((0,1)):
            qc.z(2+9*pos)
            qc.z(5+9*pos)
            qc.z(8+9*pos)
            
    return qc

qc = rot_surf_code(1)
ancilla = qc.num_qubits - 2

qc = Hadamard(qc,0)

for i in range(4):
    qc = T_L_test(circuit = qc, had = True, pos = 0)

#qc = Z_L(qc, True, 0)

qc = Hadamard(qc,0)

qc.reset(ancilla)

qc = Z_parity(qc,False,0)

qc.measure(ancilla,0)

sim = AerSimulator()
job = sim.run(qc, shots=1000)
result = job.result()
counts = result.get_counts()

print("Wir wollen _ _ _ 1: ", counts)
gates(qc)

Wir wollen _ _ _ 1:  {'0001': 1000}
Amount of gates in this circuit:  87


In [86]:
qc = rot_surf_code(1)
ancilla = qc.num_qubits - 2
magic = qc.num_qubits - 1

qc = Hadamard(qc,0)

had = True
pos = 0

qc.h(magic)
qc.t(magic)

qc = Z_parity(qc,True,0)
qc.cx(magic, ancilla)

qc.measure(ancilla,0)

if had == False:
    with qc.if_test((0,1)):
        qc.s(0+9*pos)
        qc.s(1+9*pos)
        qc.s(2+9*pos)
        qc.s(3+9*pos)
        qc.s(4+9*pos)
        qc.s(5+9*pos)
        qc.s(6+9*pos)
        qc.s(7+9*pos)
        qc.s(8+9*pos)

        qc.z(6+9*pos)
        qc.z(7+9*pos)
        qc.z(8+9*pos)
else:
    with qc.if_test((0,1)):
        qc.s(0+9*pos)
        qc.s(1+9*pos)
        qc.s(2+9*pos)
        qc.s(3+9*pos)
        qc.s(4+9*pos)
        qc.s(5+9*pos)
        qc.s(6+9*pos)
        qc.s(7+9*pos)
        qc.s(8+9*pos)
        
        qc.z(2+9*pos)
        qc.z(5+9*pos)
        qc.z(8+9*pos)

qc.h(magic)
qc.measure(magic,0)

if had == False:
    with qc.if_test((0,1)):
        qc.z(6+9*pos)
        qc.z(7+9*pos)
        qc.z(8+9*pos)
else:
    with qc.if_test((0,1)):
        qc.z(2+9*pos)
        qc.z(5+9*pos)
        qc.z(8+9*pos)



sim = AerSimulator()
job = sim.run(qc, shots=999)
result = job.result()
counts = result.get_counts()

print("Wir wollen _ _ _ 1: ", counts)
gates(qc)

Wir wollen _ _ _ 1:  {'0000': 527, '0001': 472}
Amount of gates in this circuit:  32
